# Assignment 1 Network Analytics
**Group 5:** Shaked Atia, Achyuthuni Harsha, Ahmed Khedr, Ankit Mahajan  

**NOTE: The whole document uses code snippets and text from the Gourobi documentation of Travelling salesmen problem. https://www.gurobi.com/resource/traveling-salesman-problem/**

The given file is in text format with the relavant data starting with a numeric value. Reading the data to a dataframe, we have: 

In [1]:
import pandas as pd
import numpy as np
file = open("HW2_qatar_tsp.txt", "r")
# Reading the lines that have  proper latitude and longitude value into a dataframe

# Creating an empty datafrae to store the data
df = pd.DataFrame({'id':[], 'x_cord':[], 'y_cord':[]})
for x in file:
    if(x[0].isnumeric()):
        df.loc[len(df.index)] = x.split()
df = df.astype(float)
df

,id,x_cord,y_cord
0,1.0,24748.3333,50840.0000
1,2.0,24758.8889,51211.9444
2,3.0,24827.2222,51394.7222
3,4.0,24904.4444,51175.0000
4,5.0,24996.1111,51548.8889
...,...,...,...
189,190.0,26123.6111,51169.1667
190,191.0,26123.6111,51222.7778
191,192.0,26133.3333,51216.6667
192,193.0,26133.3333,51300.0000


In the above data, the latitudes and longitudes are provided (multiplied by 1000) for various cities in the country of Qatar. This can be shown as a scatter plot (with a background of Qatar) in the below plot.

In [34]:
import plotly.express as px
df['lat'] = df.x_cord/1000
df['long'] = df.y_cord/1000

# Using the plotly express package which has the function scatter geo to plot the map of qatar
fig = px.scatter_geo(df,lat='lat',lon='long', fitbounds = 'locations')
fig.update_layout(title = 'Map of Qatar', title_x=0.5)
fig.show()

In case the above plot is not visible in the pdf/jupyter version, please refer the file 'Qatar dot plot.png' for the plot.   
Below we are creating a dictionary containing the x and y coordinates of the different cities in the dataset.

In [3]:
cities = list(df.id)
coordinates = {}
for location in cities:
    coordinates[location] = (float(df.loc[df['id'] == location]['x_cord'])/1000, float(df.loc[df['id'] == location]['y_cord'])/1000)
coordinates

{1.0: (24.7483333, 50.84),
 2.0: (24.758888900000002, 51.2119444),
 3.0: (24.8272222, 51.3947222),
 4.0: (24.9044444, 51.175),
 5.0: (24.996111099999997, 51.5488889),
 6.0: (25.01, 51.0394444),
 7.0: (25.030833299999998, 51.275277800000005),
 8.0: (25.067777799999998, 51.0775),
 9.0: (25.1, 51.5166667),
 10.0: (25.1033333, 51.521666700000004),
 11.0: (25.1219444, 51.2183333),
 12.0: (25.1508333, 51.5377778),
 13.0: (25.1583333, 51.163611100000004),
 14.0: (25.1622222, 51.220833299999995),
 15.0: (25.1677778, 51.6069444),
 16.0: (25.168888900000002, 51.086388899999996),
 17.0: (25.1738889, 51.2694444),
 18.0: (25.210833299999997, 51.3941667),
 19.0: (25.211388900000003, 51.6191667),
 20.0: (25.2141667, 50.8072222),
 21.0: (25.2144444, 51.3788889),
 22.0: (25.2233333, 51.451666700000004),
 23.0: (25.2241667, 51.1744444),
 24.0: (25.233333299999998, 51.3333333),
 25.0: (25.234166700000003, 51.2030556),
 26.0: (25.2355556, 51.33),
 27.0: (25.2355556, 51.4955556),
 28.0: (25.2427778, 51.428

## Define distances
As these are locations, we can take calculate distances based on certain assumptions.  
1. We can assume that the earth is flat and compute distances between two locations using euclidian distances. This would be reasonable for a small country like Qatar, which could be approximately considered flat on the scale of the earth. Also it wouldnt significantly change the output.  
2. We could assume that the earth is a sphere and compute distances.
3. The earth is not actually a sphere and is only approximated to one. We could calculate distances by taking the local curvature of earth also. We are not doing this in the current implementation.

In [4]:
import math
from itertools import combinations

# Compute pairwise distance matrix

def distance(city1, city2):
    c1 = coordinates[city1]
    c2 = coordinates[city2]
    diff = (c1[0]-c2[0], c1[1]-c2[1])
    return math.sqrt(diff[0]*diff[0]+diff[1]*diff[1])

dist = {(c1, c2): distance(c1, c2) for c1, c2 in combinations(cities, 2)}

### Optimiser code

Consider $x_{i,j}$ to be the edge between city $i$ and city $j$. If the route between the edge has to be taken it should be 1 and if not it should be 0. Using Gourobi, we can write linear (and integer) constraints and objective function. Because this is the _symmetric_ traveling salesman problem, we can make it more efficient by setting the _object_ x[j,i] to x[i,j], instead of a constraint.

In [5]:
import gurobipy as gp
from gurobipy import GRB

# tested with Python 3.7 & Gurobi 9.0.0

m = gp.Model()

# Variables: is city 'i' adjacent to city 'j' on the tour?
vars_ = m.addVars(dist.keys(), obj=dist, vtype=GRB.BINARY, name='e')

# Symmetric direction: Copy the object
for i, j in vars_.keys():
    vars_[j, i] = vars_[i, j]  # edge in opposite direction

# Constraints: two edges incident to each city
cons = m.addConstrs(vars_.sum(c, '*') == 2 for c in cities)

Academic license - for non-commercial use only - expires 2021-08-11
Using license file C:\Users\Achyuthuni\gurobi.lic


### Objective Function 
Reference: https://www.gurobi.com/resource/traveling-salesman-problem/  

- **Shortest Route**. Minimize the total distance of a route.

\begin{equation}
\text{Min} \quad Z = \sum_{(i,j) \in \text{Pairings}}d_{i,j} \cdot x_{i,j}
\tag{0}
\end{equation}

### Constraints 
- **Symmetry Constraints**. For each edge $(i,j)$, ensure that the cities $i$ and $j$ are connected, if the former is visited immediately before or after visiting the latter.

\begin{equation}
x_{i, j} = x_{j, i} \quad \forall (i, j) \in Pairings
\tag{1}
\end{equation}

- **Entering and leaving a city**. For each capital city $i$, ensure that this city is connected to two other cities. 

\begin{equation}
\sum_{(i,j) \in \text{Pairings}}x_{i,j} = 2 \quad \forall  i \in Cities
\tag{2}
\end{equation}

- **Subtour elimination**. These constraints ensure that for any subset of cities $S$, there is no cycle. That is, there is no route that visits all the cities in the subset and returns to the origin city.

\begin{equation}
\sum_{(i \neq j) \in S}x_{i,j} \leq |S|-1 \quad \forall  S \subset  Cities
\tag{3}
\end{equation}

  
This subtour elimination is implemented sequentially after every run until we have a only one route.

In [6]:
# Callback - use lazy constraints to eliminate sub-tours

def subtourelim(model, where):
    if where == GRB.Callback.MIPSOL:
        # make a list of edges selected in the solution
        vals = model.cbGetSolution(model._vars)
        selected = gp.tuplelist((i, j) for i, j in model._vars.keys()
                             if vals[i, j] > 0.5)
        # find the shortest cycle in the selected edge list
        tour = subtour(selected)
        if len(tour) < len(cities):
            # add subtour elimination constr. for every pair of cities in subtour
            model.cbLazy(gp.quicksum(model._vars[i, j] for i, j in combinations(tour, 2))
                         <= len(tour)-1)

# Given a tuplelist of edges, find the shortest subtour

def subtour(edges):
    unvisited = cities[:]
    cycle = cities[:] # Dummy - guaranteed to be replaced
    while unvisited:  # true if list is non-empty
        thiscycle = []
        neighbors = unvisited
        while neighbors:
            current = neighbors[0]
            thiscycle.append(current)
            unvisited.remove(current)
            neighbors = [j for i, j in edges.select(current, '*')
                         if j in unvisited]
        if len(thiscycle) <= len(cycle):
            cycle = thiscycle # New shortest subtour
    return cycle

## Solving the model

In [7]:
m._vars = vars_
m.Params.lazyConstraints = 1
m.optimize(subtourelim)

# Validate the solution
vals = m.getAttr('x', vars_)
selected = gp.tuplelist((i, j) for i, j in vals.keys() if vals[i, j] > 0.5)

tour = subtour(selected)
assert len(tour) == len(cities)

Changed value of parameter lazyConstraints to 1
   Prev: 0  Min: 0  Max: 1  Default: 0
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 194 rows, 18721 columns and 37442 nonzeros
Model fingerprint: 0xf2d24ad7
Variable types: 0 continuous, 18721 integer (18721 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e-03, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+00, 2e+00]
Presolve time: 0.09s
Presolved: 194 rows, 18721 columns, 37442 nonzeros
Variable types: 0 continuous, 18721 integer (18721 binary)

Root relaxation: objective 9.006487e+00, 302 iterations, 0.11 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    9.00649    0   48          -    9.00649      -     -    0s
     0     0    9.11875    0   30      

## Analysis

We retrieve the optimal solution of the TSP and verified that the optimal route (or tour) goes to all the cities and returns to the origin city.

In [8]:
print('Optimal tour: %s' % str(tour))
print('Optimal cost: %g' % (m.objVal*1000))

Optimal tour: [1.0, 4.0, 2.0, 3.0, 5.0, 9.0, 10.0, 12.0, 15.0, 19.0, 30.0, 32.0, 31.0, 35.0, 42.0, 50.0, 55.0, 49.0, 44.0, 38.0, 41.0, 46.0, 48.0, 54.0, 52.0, 53.0, 56.0, 58.0, 43.0, 40.0, 34.0, 39.0, 47.0, 51.0, 61.0, 67.0, 73.0, 66.0, 68.0, 64.0, 70.0, 77.0, 84.0, 81.0, 79.0, 83.0, 88.0, 92.0, 97.0, 95.0, 96.0, 93.0, 91.0, 103.0, 102.0, 109.0, 113.0, 114.0, 119.0, 122.0, 118.0, 106.0, 105.0, 107.0, 108.0, 100.0, 110.0, 112.0, 115.0, 116.0, 117.0, 121.0, 120.0, 123.0, 124.0, 128.0, 133.0, 135.0, 129.0, 131.0, 136.0, 143.0, 148.0, 160.0, 166.0, 171.0, 185.0, 193.0, 188.0, 189.0, 191.0, 192.0, 190.0, 187.0, 183.0, 186.0, 194.0, 182.0, 176.0, 169.0, 161.0, 163.0, 164.0, 172.0, 179.0, 174.0, 173.0, 175.0, 184.0, 177.0, 181.0, 178.0, 180.0, 170.0, 167.0, 168.0, 165.0, 159.0, 158.0, 162.0, 155.0, 151.0, 147.0, 152.0, 141.0, 144.0, 150.0, 153.0, 157.0, 154.0, 139.0, 138.0, 142.0, 146.0, 149.0, 156.0, 145.0, 140.0, 137.0, 134.0, 132.0, 126.0, 125.0, 127.0, 130.0, 111.0, 104.0, 101.0, 99.0, 94

The optimal route is displayed in the following map. (Please zoom into the area of Qatar, if the image is not clear, please find the image as a png file in the attachments)

In [33]:
# Map the solution
# Reference: https://georgetsilva.github.io/posts/mapping-points-with-folium/

import folium

map = folium.Map(location=[25.5,51.5], zoom_start = 8)

points = []
for city in tour:
    points.append(coordinates[city])
points.append(points[0])

folium.PolyLine(points).add_to(map)

map

# References
1. https://www.gurobi.com/resource/traveling-salesman-problem/ (The whole document uses code snippets and text from the Gourobi documentation of Travelling salesmen problem.)